# KNN _versus_ SGD 

A equipe segue recomendações presentes nas [documentações](http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) do sklearn.

O terceiro encontro da equipe reuniu esforços para estreitar os métodos de classificação a serem utilizados que trouxessem uma maior acurácia quanto aos resultados. Nesse sentido, primeiramente foi necessário analisar a base de dados, conhecendo a dimensão desta, de modo que observou-se um total de 5.000.000 de registros. Constatou-se que o problema apresentado girava em torno da tarefa de predizer uma categoria por meio de dados que apresentam labels, ou seja, um problema de classificação. 

Pela dimensão do banco optou-se testou-se _Support Vector Classification_ ou SVC, mas não obteve-se sucesso, além disso, não envolvia problemas relacionados a texto, por tanto, a equipe não testou Naive Bayes, partindo então para _Nearest Neighbors Classification_. A princípio mostrou resultados razoáveis, como é possível ver [aqui](#knnAcuracia), entretanto, o número de dados processados não englobava todos os registros do dataset. Por meio do [tutorial](http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) do scikitlearn, percebeu-se que esse método não era recomendado para conjunto de dados maiores que 100k.

A incompatibilidade relatada, pode ser observada devido ao fato de que com acréscimos no número de registros a serem processados o algorítmo se tornava demasiadamente lento, como pode-se testar e observar na seção [Tempo de Execução](#tempoExecucao). É importante salientar que a análise desse algorítmo foi realizada pela equipe considerando apenas as últimas 8 colunas, pois a equipe raciocinou que, como elas eram funções das 10 primeiras poderia apresentar uma maior independência entre si, os resultados apresentados aqui, consideram essa condição. A partir de então, a equipe optou por testar [SGDClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html).


## Nearest Neighbors Classification

A classificação baseada em vizinhos é um tipo de aprendizado que se baseia em intâncias : não tenta construir um modelo interno geral, mas simplesmente armazena instâncias dos dados de treinamento. A classificação é calculada a partir de uma votação simples dos vizinhos mais próximos de cada ponto.

A classificação básica de vizinhos mais próximos usa pesos uniformes: o valor atribuído a um ponto é calculado a partir de uma votação simples por maioria  dos vizinhos mais próximos. Em algumas circunstâncias, é melhor ponderar os vizinhos de tal forma que os vizinhos mais próximos contribuam mais para o ajuste.

[Documentação](http://scikit-learn.org/stable/modules/neighbors.html#classification)


## KNeighborsClassifier

É um algorítmo classificador que implementa o voto entre os k vizinhos mais próximos. 

[Documentação](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

## SGD - Stochastic Gradient Descent

O Algorítmo SGD é uma abordagem simples porém eficiente, sendo esta abordagem, facilmente implementada. A abordagem SGD é utilizada para modelos discriminativos com classificadores lineares sobre funções convexas de perda, tais como a regressão linear.Bastante aplicado em aprendizado de larga escala, o SGDClassifier é a alternativa para bases de dados em que o KNeighborsClassifier se mostra ineficiente.

Por trabalhar com bases de dados de grande volume, o SGD necessita de hiper parametros, ou seja, necessita que sejam analisadas as features mais relevantes. Isso se alcança com uma boa feature selection antes de se aplicar o SGD Classifier.

[Documentação](http://scikit-learn.org/stable/modules/sgd.html)

## SGDClassifier

É um algorítmo classificador que implementa modelos lineares utilizando curvas de gradiente.

[documentacao](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)

## Importando os módulos necessários



In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import VarianceThreshold

## Importando os dados do arquivo csv (apenas as 100000 observações finais do arquivo)

In [2]:
t = np.genfromtxt('SUSY.csv', delimiter=',', usecols=(9,10,11,12,13,14,15,16,17,18), skip_footer=4900000)
v = np.genfromtxt('SUSY.csv', delimiter=',', usecols=(0), skip_footer=4900000)

## Seletor de features que remove todas as que possuem baixa variação

[Documentação VarianceThreshold](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html)

In [3]:
#threshold : Limite de variância a partir da qual as features serão removidas

sel = VarianceThreshold(threshold=(.9 * (1 - .9)))
dataset_susy = sel.fit_transform(t)

## Define parcela de treino e de teste dos dados (30% teste, 70% treino)


In [6]:
dataset_susy[0,:]

t_treino, t_teste, v_treino, v_teste = train_test_split(dataset_susy, v, test_size=0.3, random_state=0)


## Testa o KNeighborsClassifier variando o número de vizinhos para comparar a acurácia 
<a name="knnAcuracia"></a>

In [7]:
for i in range(1, 19):
    knn = KNeighborsClassifier(n_neighbors=i, p=2)
    knn.fit(t_treino, v_treino)
    labels = knn.predict(t_teste)
    print("{} : {}".format(i, knn.score(t_teste, v_teste)))

1 : 0.7100666666666666
2 : 0.7316333333333334
3 : 0.7428333333333333
4 : 0.7538666666666667
5 : 0.7570666666666667
6 : 0.7621666666666667
7 : 0.7650666666666667
8 : 0.7683666666666666
9 : 0.7690666666666667
10 : 0.7701
11 : 0.7734333333333333
12 : 0.7731
13 : 0.7745
14 : 0.7739333333333334
15 : 0.7762
16 : 0.7758333333333334
17 : 0.7783333333333333
18 : 0.7772333333333333


# Marcando tempo de execução com diferentes tamanhos de dataset

<a name="tempoExecucao"></a>

In [12]:
#Importando módulos necessários (SE não foi feito anteriormente)

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import VarianceThreshold
import time

In [5]:
# Valor inicial de 100.000 registros, para testar outras quantidades basta alterar skip_footer : 5 milhões - valorDesejado

t = np.genfromtxt('SUSY.csv', delimiter=',', usecols=(9,10,11,12,13,14,15,16,17,18), skip_footer=4900000)
v = np.genfromtxt('SUSY.csv', delimiter=',', usecols=(0), skip_footer=4900000)


In [13]:
# Executar para 500.000 registros

t = np.genfromtxt('SUSY.csv', delimiter=',', usecols=(9,10,11,12,13,14,15,16,17,18), skip_footer=4500000)
v = np.genfromtxt('SUSY.csv', delimiter=',', usecols=(0), skip_footer=4500000)

In [14]:
#Prepara treino e Teste

sel = VarianceThreshold(threshold=(.9 * (1 - .9)))
dataset_susy = sel.fit_transform(t)
dataset_susy[0,:]

t_treino, t_teste, v_treino, v_teste = train_test_split(dataset_susy, v, test_size=0.3, random_state=0)



# Tempo de execução com 100.000 registros(teste para i vizinhos)

In [8]:

for i in range(1, 19):
    inicio = time.time()
    knn = KNeighborsClassifier(n_neighbors=i, p=2)
    knn.fit(t_treino, v_treino)
    labels = knn.predict(t_teste)
    fim = time.time()
    print("{} : {}, tempo de execução (em segundos): {}".format(i, knn.score(t_teste, v_teste), (fim-inicio)))

1 : 0.7100666666666666, tempo de execução (em segundos): 1.6862399578094482
2 : 0.7316333333333334, tempo de execução (em segundos): 2.1964118480682373
3 : 0.7428333333333333, tempo de execução (em segundos): 1.7621791362762451
4 : 0.7538666666666667, tempo de execução (em segundos): 1.7782456874847412
5 : 0.7570666666666667, tempo de execução (em segundos): 1.981682538986206
6 : 0.7621666666666667, tempo de execução (em segundos): 2.13596510887146
7 : 0.7650666666666667, tempo de execução (em segundos): 2.179724931716919
8 : 0.7683666666666666, tempo de execução (em segundos): 2.2925169467926025
9 : 0.7690666666666667, tempo de execução (em segundos): 2.474717140197754
10 : 0.7701, tempo de execução (em segundos): 2.6374094486236572
11 : 0.7734333333333333, tempo de execução (em segundos): 2.677070140838623
12 : 0.7731, tempo de execução (em segundos): 2.5869197845458984
13 : 0.7745, tempo de execução (em segundos): 2.668410062789917
14 : 0.7739333333333334, tempo de execução (em segu

# Tempo de execução com 200.000 registros(teste para i vizinhos)

In [11]:
for i in range(1, 19):
    inicio = time.time()
    knn = KNeighborsClassifier(n_neighbors=i, p=2)
    knn.fit(t_treino, v_treino)
    labels = knn.predict(t_teste)
    fim = time.time()
    print("{} : {}, tempo de execução (em segundos): {}".format(i, knn.score(t_teste, v_teste), (fim-inicio)))



1 : 0.7087666666666667, tempo de execução (em segundos): 2.818880558013916
2 : 0.7328, tempo de execução (em segundos): 3.597216844558716
3 : 0.74715, tempo de execução (em segundos): 4.110960245132446
4 : 0.7553333333333333, tempo de execução (em segundos): 4.513943195343018
5 : 0.761, tempo de execução (em segundos): 4.82697606086731
6 : 0.7662833333333333, tempo de execução (em segundos): 5.181045770645142
7 : 0.7693333333333333, tempo de execução (em segundos): 5.459136724472046
8 : 0.77105, tempo de execução (em segundos): 5.831350564956665
9 : 0.7723333333333333, tempo de execução (em segundos): 6.012509822845459
10 : 0.7739, tempo de execução (em segundos): 6.2784202098846436
11 : 0.7756833333333333, tempo de execução (em segundos): 6.643414497375488
12 : 0.7759666666666667, tempo de execução (em segundos): 6.733079433441162
13 : 0.77775, tempo de execução (em segundos): 6.849187135696411
14 : 0.77825, tempo de execução (em segundos): 7.110193252563477
15 : 0.7803833333333333, t

# Tempo de execução com 500.000 registros(teste para i vizinhos)

In [15]:
for i in range(1, 19):
    inicio = time.time()
    knn = KNeighborsClassifier(n_neighbors=i, p=2)
    knn.fit(t_treino, v_treino)
    labels = knn.predict(t_teste)
    fim = time.time()
    print("{} : {}, tempo de execução (em segundos): {}".format(i, knn.score(t_teste, v_teste), (fim-inicio)))


1 : 0.71302, tempo de execução (em segundos): 10.082895278930664
2 : 0.73694, tempo de execução (em segundos): 12.100428819656372
3 : 0.7497266666666667, tempo de execução (em segundos): 13.046036005020142
4 : 0.7585266666666667, tempo de execução (em segundos): 14.402191400527954
5 : 0.7637466666666667, tempo de execução (em segundos): 15.6648850440979
6 : 0.7684866666666667, tempo de execução (em segundos): 16.35658311843872
7 : 0.7713066666666667, tempo de execução (em segundos): 17.50118899345398
8 : 0.77428, tempo de execução (em segundos): 17.883357524871826
9 : 0.77662, tempo de execução (em segundos): 18.63050651550293
10 : 0.7780733333333333, tempo de execução (em segundos): 20.270562171936035
11 : 0.77952, tempo de execução (em segundos): 20.789804935455322
12 : 0.7805333333333333, tempo de execução (em segundos): 19.97312569618225
13 : 0.7808533333333333, tempo de execução (em segundos): 20.544463872909546
14 : 0.78182, tempo de execução (em segundos): 21.25261878967285
15 :

Nota-se a partir da célula anterior, que além de o tempo aumentar com o número de registros processados, também aumenta com o número de vizinhos a serem considerados nas interações.
